# Future Research: Proof-of-Concept Notebook

This notebook provides a concrete, runnable example for one of the ideas outlined in `FUTURE_RESEARCH.md`: **analyzing mixed CNOT orderings**.

The analyses in the other notebooks assume that all weight-4 stabilizers use the same CNOT permutation. However, the library is flexible enough to support different orderings for different stabilizers. This notebook demonstrates how to set up and run such an analysis.

In [ ]:
# === 1. Imports and Setup ===
import pyzx as zx
import pprint

from zx_error_prop.surface_code import generate_surface_code_qasm
from zx_error_prop.propagation import propagate_pauli_error, get_output_errors, count_logical_errors

In [ ]:
# === 2. Define Mixed CNOT Orderings ===

DISTANCE = 3
Z_LOGICALS = [[73, 74, 75], [76, 77, 78], [79, 80, 81]]
X_LOGICALS = [[73, 76, 79], [74, 77, 80], [75, 78, 81]]

# Let's compare a 'good' permutation with a 'bad' one from the original analysis.
GOOD_PERM = [0, 3, 1, 2] # Known to be robust
BAD_PERM  = [0, 1, 2, 3] # Known to cause logical errors

# A hook error known to cause issues with the bad permutation
HOOK_ERROR = [((37, 35), 'X')] # X-hook on stabilizer X1

# Case 1: All stabilizers use the BAD permutation
bad_orders = {
    'Z0': BAD_PERM,
    'X1': BAD_PERM,
    'X2': BAD_PERM,
    'Z3': BAD_PERM
}

# Case 2: All stabilizers use the GOOD permutation
good_orders = {
    'Z0': GOOD_PERM,
    'X1': GOOD_PERM,
    'X2': GOOD_PERM,
    'Z3': GOOD_PERM
}

# Case 3: A MIXED ordering. Let's make only the target stabilizer robust.
mixed_orders = {
    'Z0': BAD_PERM, 
    'X1': GOOD_PERM, # The stabilizer we are injecting an error on
    'X2': BAD_PERM,
    'Z3': BAD_PERM
}

In [ ]:
# === 3. Run and Compare Analyses ===

def run_single_analysis(name, distance, ordering_dict, error_to_inject):
    """Helper function to run the analysis for a single configuration."""
    print(f"--- Running Analysis for: {name} ---")
    
    # Generate the circuit
    qasm_string = generate_surface_code_qasm(distance, custom_cnot_orderings=ordering_dict)
    graph = zx.sqasm(qasm_string, simplify=False)
    
    # Propagate the error
    final_web, _ = propagate_pauli_error(graph, error_to_inject)
    output_errors = get_output_errors(graph, final_web)
    
    # Count logical errors
    z_errs, x_errs = count_logical_errors(output_errors, Z_LOGICALS, X_LOGICALS)
    
    print(f"Result: Z Errors = {z_errs}, X Errors = {x_errs}")
    if z_errs > 0 or x_errs > 0:
        print("Conclusion: A logical error occurred.")
    else:
        print("Conclusion: No logical error occurred.")
    print("\n")

# Run the comparison
run_single_analysis("All Bad Permutation", DISTANCE, bad_orders, HOOK_ERROR)
run_single_analysis("All Good Permutation", DISTANCE, good_orders, HOOK_ERROR)
run_single_analysis("Mixed Permutation", DISTANCE, mixed_orders, HOOK_ERROR)

### Conclusion

This proof-of-concept demonstrates that the library can be used to explore more complex CNOT ordering schemes. By modifying the `custom_orders` dictionary, a researcher can easily test any combination of permutations for any set of stabilizers, providing a powerful tool for investigating the fine-grained details of error propagation in these codes.